In [1]:
#Declare packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import os

In [2]:
path = os.getcwd()
print(str(path).replace("\/", "/"))
os.chdir(path)

c:\Users\HP\Pictures\g\src\analysis


In [3]:
#df = pd.read_csv('C:/Users/HP/Pictures/HR/HR-Analytics/data/raw/HR-Employee-Attrition.csv')
df = pd.read_csv("C:/Users/HP/Pictures/g/data/processed/train_data_attrition.csv")
# df=pd.read_csv("/content/HR-Employee-Attrition.csv")
df.head()

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,24,Travel_Rarely,350,Research & Development,21,2,Technical Degree,1,1551,3,...,80,3,2,3,3,1,1,0,0,No
1,18,Non-Travel,287,Research & Development,5,2,Life Sciences,1,1012,2,...,80,0,0,2,3,0,0,0,0,No
2,29,Travel_Rarely,1247,Sales,20,2,Marketing,1,349,4,...,80,1,10,2,3,3,2,0,2,No
3,39,Travel_Rarely,492,Research & Development,12,3,Medical,1,1654,4,...,80,0,7,3,3,5,4,1,0,No
4,31,Travel_Rarely,311,Research & Development,20,3,Life Sciences,1,1881,2,...,80,1,10,2,3,10,8,0,2,No


# Exploration Data Analysis

Xem xét quy mô công ty

In [4]:
fig=make_subplots(rows=1, cols=2,
                  subplot_titles=("", "Số lượng nhân viên nghỉ việc theo phòng ban"),
                  specs=[[{"type": "bar"}, {"type": "pie"}]])

# Bar chart
plot_df=df['Attrition'].value_counts(normalize=True)
plot_df=plot_df.mul(100).rename('Percent').reset_index().sort_values('Percent')
plot_df.rename(columns={'index':'Attrition'}, inplace=True)
plot_df['Attrition']=['Cựu nhân viên' if i == 'Yes' else 'Nhân viên hiện tại' for i in plot_df['Attrition']]
x=plot_df['Attrition']
y=plot_df['Percent']
fig.add_trace(
    go.Bar(x=x, y=y, text=y,opacity=.8,
           hovertemplate='Employee Attrition Rate<br>%{x}: %{y:.3}%<extra></extra>',
           showlegend=False), row=1, col=1)
fig.update_traces(texttemplate='%{text:.3s}%', textposition='outside',
                  marker_line=dict(width=1, color='#1F0202'), marker_color=["#C41823","#368EED"])
fig.update_yaxes(zeroline=True, zerolinewidth=1, zerolinecolor='gray')
fig.update_layout(yaxis_ticksuffix = '%')

# Pie chart
plot_df2=df[df.Attrition=='Yes']
plot_df2=plot_df2['Department'].value_counts(normalize=True)
plot_df2=plot_df2.mul(100).rename('Percent').reset_index().sort_values('Percent', ascending=False)
plot_df2.rename(columns={'index':'Department'}, inplace=True)
fig.add_trace(go.Pie(labels=plot_df2['Department'], values=plot_df2['Percent'], opacity=0.85, hole=0.4,
                     hovertemplate='%{label}<br>Attrition Rate: %{value:.3}%<extra></extra>',
                     marker_colors=["#A70C09","#E74824","#D08841"]), row=1, col=2)
                    #  marker_colors=['#587D65','#ADC4B2','#D1C9C2']), row=1, col=2)
fig.update_yaxes(tickmode = 'array', range=[0, 90], dtick=5)
fig.update_traces(textfont_size=14,textfont_color='black',marker=dict(line=dict(color='#28221D', width=1)))
fig.update_layout(title_text="Thống kê về tình trạng làm việc tại công ty", font_color='#28221D',
                  paper_bgcolor='#F4F2F0', plot_bgcolor='#F4F2F0')
fig.show()

- Dựa vào biểu đồ ta thấy trong bộ dữ liệu bao gồm 16.1% nhân viên đã nghỉ việc và có 83.9% nhân viên vẫn còn đang làm
- Trong đó
    + Ở phòng ban R&D, số lượng nhân viên nghỉ việc chiếm tỷ lệ cao nhất, khoảng 51.3%
    + Ngay sau đó là phòng ban Sales, chiếm khoảng 38.8% tỷ lệ nhân viên nghỉ việc
    + Phòng ban HR chiếm tỷ lệ ít nhất, với 5.06%

In [5]:
age_att=df.groupby(['Age','Attrition']).apply(lambda x:x['DailyRate'].count()).reset_index(name='Counts')
fig=px.line(age_att,x='Age',y='Counts',color='Attrition',width=1000,labels={"Age":"Tuổi","Counts":"Số lượng", "Attrition":"Tình trạng nghỉ việc"},title='Số lượng nhân viên trong công ty theo độ tuổi ')
fig.for_each_trace(lambda trace: trace.update(name='Đang làm việc') if trace.name == 'No' else trace.update(name='Nghỉ việc'))
fig.show()

- Ta có thể thấy ở độ tuổi 26 đến 38 số lượng nhân viên nghỉ việc tăng cao. Điều này là vì đây là độ tuổi bắt đầu tìm kiếm việc làm nên họ thường có xu hướng muốn khám phá và tìm kiếm nhiều cơ hội mới, và họ yêu cầu trải nghiệm trong công việc nhiều hơn.
- Còn từ độ tuổi 40 trở về sau số lượng nhân viên nghỉ việc càng giảm. Bởi vì đây là độ tuổi mà họ cần có sự ổn định trong công việc để lo cho cuộc sống và đây cũng là độ tuổi có rủi ro tìm việc cao nhất

In [6]:
rate_att=df.groupby(['MonthlyIncome','Attrition']).apply(lambda x:x['MonthlyIncome'].count()).reset_index(name='Counts')
rate_att['MonthlyIncome']=round(rate_att['MonthlyIncome'],-3)
rate_att=rate_att.groupby(['MonthlyIncome','Attrition']).apply(lambda x:x['MonthlyIncome'].count()).reset_index(name='Counts')
fig=px.line(rate_att,x='MonthlyIncome',y='Counts',color='Attrition',width=1000,labels={"MonthlyIncome":"Thu nhập hàng tháng","Counts":"Số lượng","Attrition":"Tình trạng nghỉ việc"},title='Số lượng nhân viên theo thu nhập hàng tháng trong công ty   ')
fig.for_each_trace(lambda trace: trace.update(name='Đang làm việc') if trace.name == 'No' else trace.update(name='Nghỉ việc'))
fig.show()

- Ta có thể thấy ở mức lương thấp (<5k) số lượng nhân viên nghỉ việc cao và mức lương càng cao thì số lượng nhân viên nghỉ việc càng thấp
- Tuy nhiên ở mức lương 10k có sự gia tăng đột ngột, điều này có thể là vì họ đã có mức sống tốt hơn và muốn tìm một công việc khác để có tìm cơ hội tốt hơn

In [7]:
plot_df = df.groupby('JobRole')['MonthlyIncome'].mean()
plot_df = plot_df.mul(12).rename('Salary').reset_index().sort_values('Salary', ascending=False)
fig = px.bar(plot_df, x='JobRole', y='Salary', text='Salary', opacity=0.7)
fig.update_traces(texttemplate='$%{text:,.0f}', textposition='outside',
                  marker_line=dict(width=1, color='#28221D'), marker_color="#3497DE")
fig.update_yaxes(zeroline=True, zerolinewidth=1, zerolinecolor='#28221D')
fig.update_layout(title_text='Lương trung bình theo vai trò trong công ty', font_color='#28221D',
                  yaxis=dict(title='Salary',tickprefix='$'), height=500,
                  xaxis_title='', paper_bgcolor='#F4F2F0', plot_bgcolor='#F4F2F0')
fig.show()

Nhận xét: Ta thấy những người giữ chức vụ quan trọng trong công ty sẽ có những mức đãi ngộ trung bình cao hơn (đối với vị trí Manager là >200k, Reseach Director là >190k), ngoài ra những vị trí khác cũng có mức lương tương đối cao (Healthcare Representative viws ~90k, Manufacturing Director với ~87k, Sales Executive với ~ 83k)

Thông thường trong một số công ty, nhân viên sẽ giữ một phần cổ phiếu của công ty, và trong bộ dữ liệu này ta có trường 'StockOptionLevel' thể hiện sự sở hữu cổ phiếu của công ty của nhân viên theo cấp bậc

In [8]:
dept_sales = df[df['Department'] == 'Sales'].groupby('Attrition').apply(lambda x: x['DailyRate'].count()).reset_index(name='Counts')
dept_rnd = df[df['Department'] == 'Research & Development'].groupby('Attrition').apply(lambda x: x['DailyRate'].count()).reset_index(name='Counts')
dept_hr = df[df['Department'] == 'Human Resources'].groupby('Attrition').apply(lambda x: x['DailyRate'].count()).reset_index(name='Counts')
fig = make_subplots(rows=1, cols=3,
                    specs=[[{'type': 'pie'}, {'type': 'pie'}, {'type': 'pie'}]],
                    subplot_titles=['Tỉ lệ nghỉ việc trong phòng Kinh doanh', 'Tỉ lệ nghỉ việc trong phòng Nghiên cứu và Phát triển', 'Tỉ lệ nghỉ việc trong phòng Nhân sự'])
fig.add_trace(
    px.pie(dept_sales, names='Attrition', values='Counts',labels={"Attrition":"Tình trạng nghỉ việc","Counts":"Số lượng"}).data[0],
    row=1, col=1
)

fig.add_trace(
    px.pie(dept_rnd, names='Attrition', values='Counts',labels={"Attrition":"Tình trạng nghỉ việc","Counts":"Số lượng"}).data[0],
    row=1, col=2
)

fig.add_trace(
    px.pie(dept_hr, names='Attrition', values='Counts',labels={"Attrition":"Tình trạng nghỉ việc","Counts":"Số lượng"}).data[0],
    row=1, col=3
)
fig.update_layout(
    title_text="Tỉ lệ nghỉ việc theo từng phòng ban",
    height=400, width=1400,
    showlegend=True
)
fig.update_traces(
    selector=dict(type='pie'),
    labels=['Đang làm việc' if label == 'No' else 'Nghỉ việc' for label in ['No', 'Yes']],marker=dict(colors=['#3497DE', '#DE1515']
))
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

Dựa vào các biểu đồ trên ta thấy:
- Phòng có tỉ lệ nghỉ việc nhiều nhất là phòng Kinh danh chiểm 20,6% trong tổng số 446 nhân viên.
- Kế tiếp là phòng Nhân sự chiếm 19% trong tổng số 63 nhân viên
- Và cuối cùng là phòng Nghiên cứu và Phát triển chiếm 13,8%  trong tổng số 961 nhân viên

In [9]:
stock_att=df.groupby(['StockOptionLevel','Attrition']).apply(lambda x:x['DailyRate'].count()).reset_index(name='Counts').replace({'No': 'Đang làm việc', 'Yes': 'Nghỉ việc'})
fig = px.bar(stock_att, x='StockOptionLevel', y='Counts', color='Attrition',
             title='Số lượng nhân viên nắm giữ cổ phiếu công ty theo phân cấp',
             width=1000,
             color_discrete_map={'Đang làm việc': '#3497DE', 'Nghỉ việc': "#DE1515"})

fig.update_layout(
    xaxis_title="Mức Sở Hữu Cổ Phiếu",
    yaxis_title="Số Lượng"
)

fig.update_traces(
    texttemplate='%{y:,.0f}',
    textposition='outside',
    marker_line=dict(width=1, color='#28221D')
)

fig.show()

- Nhận xét:
Dựa vào biểu đồ, ta nhận thấy được đa số những nhân viên nghỉ việc thường nằm ở bậc 0 và 1, đây là những nhóm có sở hữu cổ phiếu bị hạn chế. Có thể thấy những người ít sở hữu cổ phần công ty có xu hướng nghỉ việc nhiều hơn.

In [10]:
sats_att=df.groupby(['EnvironmentSatisfaction','Attrition']).apply(lambda x:x['DailyRate'].count()).reset_index(name='Counts')
fig=px.area(sats_att,x='EnvironmentSatisfaction',y='Counts',color='Attrition',labels={"EnvironmentSatisfaction":"Mức độ hài lòng về môi trường làm việc","Counts":"Số lượng","Attrition":"Tình trạng nghỉ việc"},width=1200,title='Số lượng nhân viên theo mức độ hài lòng về nôi trường làm việc trong công ty')
fig.for_each_trace(lambda trace: trace.update(name='Đang làm việc') if trace.name == 'No' else trace.update(name='Nghỉ việc'))
fig.show()

In [11]:
jsats_att=df.groupby(['JobSatisfaction','Attrition']).apply(lambda x:x['DailyRate'].count()).reset_index(name='Counts')
fig=px.area(jsats_att,x='JobSatisfaction',y='Counts',color='Attrition',labels={"JobSatisfaction":"Mức độ hài lòng về công việc","Counts":"Số lượng","Attrition":"Tình trạng nghỉ việc"},width=1200,title='Số lượng nhân viên theo mức độ hài lòng về công việc trong công ty')
fig.for_each_trace(lambda trace: trace.update(name='Đang làm việc') if trace.name == 'No' else trace.update(name='Nghỉ việc'))
fig.show()

Liệu rằng yếu tố Cân bằng giữa Công việc có tác động đến tỉ lệ nghỉ việc của nhân viên tại không ty không, ta sẽ thực hiện một số trực quan để kiểm tra nhận định này

In [12]:
wlb_att=df.groupby(['WorkLifeBalance','Attrition']).apply(lambda x:x['DailyRate'].count()).reset_index(name='Counts').replace({'No': 'Đang làm việc', 'Yes': 'Nghỉ việc'})
fig = px.bar(wlb_att, x='WorkLifeBalance', y='Counts', color='Attrition',
             title='Biểu đồ thể hiện mức độ Cân bằng Công việc và Cuộc sống theo số lượng nhân viên',
             width=1000,
             color_discrete_map={'Đang làm việc': "#2792DE", 'Nghỉ việc': "#F30909"})

fig.update_layout(
    xaxis_title="Mức Cân Bằng Công Việc và Cuộc Sống",
    yaxis_title="Số Lượng"
)

fig.update_traces(
    texttemplate='%{y:,.0f}',
    textposition='outside',
    marker_line=dict(width=1, color='#28221D')
)

fig.show()

- Nhận xét:
Dựa vào biểu đồ ta thấy những người nằm ở mức thấp thường có xu hướng nghỉ việc, đối mới những người ở bậc cao hơn (bậc 3) xu hướng họ nghỉ việc nhiều hơn, có thể họ mong muốn tìm thêm cơ hội và đãi ngộ xứng đáng hơn, tuy nhiên biểu đồ này cho mức độ cân bằng cuộc sống ở mức 3 trở lên vẫn chiếm số lượng nhiều hơn, thấy nhân viên ở đây có sự hài lòng về công việc hiện tại ở công ty. Để kiểm tra nhận định này, ta sẽ tìm hiểu sự hài lòng của nhân viên đối với công ty như thế nào

# Tiếp theo ta sẽ phân tích kỹ hơn đối với nhóm Thông tin Công việc của Nhân viên

In [13]:
colors = ['#2792DE','#F30909']


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

fig = make_subplots(rows=4, cols=2, 
                     subplot_titles=['EducationField', 'Department', 'JobRole', 'OverTime','JobLevel', 'JobInvolvement', 'JobSatisfaction'],
                     vertical_spacing=0.15, horizontal_spacing=0.1)

colors = {'Yes': '#F30909', 'No': '#2792DE'} 

variables = ['EducationField', 'Department', 'JobRole', 'OverTime','JobLevel', 'JobInvolvement', 'JobSatisfaction']
subplot_positions = [(1, 1), (1, 2), (2, 1), (2, 2),(3,1),(3,2),(4,1)]

for idx, var in enumerate(variables):
    row, col = subplot_positions[idx]

    counts = df.groupby([var, 'Attrition']).size().unstack(fill_value=0)

    categories = counts.index.tolist()  
    for attrition in ['No', 'Yes']:
        fig.add_trace(
            go.Bar(
                x=categories, 
                y=counts[attrition].values,
                name='Đang làm việc' if attrition == 'No' else 'Nghỉ việc',
                marker_color=colors[attrition],
                marker_line_color='black',
                marker_line_width=1,
                text=counts[attrition].values,
                showlegend=(idx == 0)
            ),
            row=row, col=col
        )

fig.update_layout(
    width=2000, height=1000, 
    showlegend=True,
    title_text="Phân tích các yếu tố nghỉ việc liên quan đến Nhóm thông tin công việc của nhân viên",
    barmode='group',
    template='plotly_white'
)

# Update axes
for row in range(1, 5):
    for col in range(1, 3):
        fig.update_xaxes(title_text='', tickangle=30, row=row, col=col)
        fig.update_yaxes(title_text='Count', row=row, col=col)

fig.show()


- Nhận xét chung: Ta nhận thấy đa số những người còn làm việc luôn nhiều hơn những người đã nghỉ việc. Để làm rõ hơn tỉ lệ của nhân viên nghỉ việc chiếm bao nhiêu phần trăm trong từng đặc trưng, ta sẽ tiếp tục đi làm rõ

In [54]:
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=[f'{cat}' for cat in sorted(df['EducationField'].unique())],
    vertical_spacing=0.05,
    horizontal_spacing=0.05,
    specs=[[{'type': 'domain'} for _ in range(3)] for _ in range(2)]
)


attrition_labels = sorted(df['Attrition'].unique())  
color_dict = {'Yes': '#F30909', 'No': '#2792DE'} 
colors = [color_dict[label] for label in attrition_labels]  


display_labels = ['Đang làm việc' if label == 'No' else 'Nghỉ việc' for label in attrition_labels] 


edu_fields = sorted(df['EducationField'].unique())

for idx, field in enumerate(edu_fields):

    row = (idx // 3) + 1  
    col = (idx % 3) + 1   
    field_data = df[df['EducationField'] == field]
  
    attrition_counts = field_data['Attrition'].value_counts()

    total = attrition_counts.sum()
    circle = [attrition_counts.get(attr, 0) / total * 100 for attr in attrition_labels]
    

    fig.add_trace(
        go.Pie(
            labels=display_labels,  
            values=circle,
            textinfo='percent',
            textposition='inside',
            hole=0,
            marker=dict(colors=colors, line=dict(color='black', width=1)),
            pull=[0.1 if i == 1 else 0 for i in range(len(display_labels))],  
            showlegend=(idx == 0) 
        ),
        row=row, col=col
    )


fig.update_layout(
    width=1800,
    height=600,
    title_text="Tỷ lệ nhân viên nghỉ việc theo Lĩnh vực Giáo dục",
    template='plotly_white'
)


for row in range(1, 3):
    for col in range(1, 4):
        fig.update_xaxes(showticklabels=False, row=row, col=col)
        fig.update_yaxes(showticklabels=False, row=row, col=col)

fig.show()

- Nhận xét: 3 lĩnh vực Human Resources, Marketing và Technical Degree có tỉ lệ nhân viên nghỉ việc cao nhất trong tổng số 6 lĩnh vực giáo dục 

In [ ]:
fig1 = make_subplots(
    rows=1, cols=3,
    subplot_titles=[f'{cat}' for cat in sorted(df['Department'].unique())[:3]],  
    vertical_spacing=0.05,
    horizontal_spacing=0.05,
    specs=[[{'type': 'domain'} for _ in range(3)]]  
)


attrition_values = sorted(df['Attrition'].unique())  
display_labels = ['Đang làm việc' if val == 'No' else 'Nghỉ việc' for val in attrition_values] 
colors = ['#66B2FF', '#F30909']  


departments = sorted(df['Department'].unique())[:3]

for idx, dept in enumerate(departments):

    field_data = df[df['Department'] == dept]
    

    attrition_counts = field_data['Attrition'].value_counts()

    total = attrition_counts.sum()
    circle = [attrition_counts.get(attr, 0) / total * 100 for attr in attrition_values]
    
   
    fig1.add_trace(
        go.Pie(
            labels=display_labels,
            values=circle,
            textinfo='percent',
            textposition='inside',
            hole=0,
            marker=dict(colors=colors, line=dict(color='black', width=1)),
            pull=[0.1 if i == 1 else 0 for i in range(len(display_labels))], 
            showlegend=(idx == 0)
        ),
        row=1, col=idx + 1 
    )


fig1.update_layout(
    width=1800,
    height=300,  
    title_text="Tỷ lệ nhân viên nghỉ việc theo Phòng ban",
    template='plotly_white'
)


for col in range(1, 4):  
    fig1.update_xaxes(showticklabels=False, row=1, col=col)
    fig1.update_yaxes(showticklabels=False, row=1, col=col)

fig1.show()

- Nhận xét: Trong 3 phòng ban:
    + Phòng Sales chiếm tỷ lệ nhân viên nghỉ việc cao nhất (~21.6%)
    + Tiếp đến là Phòng HR chiếm tỷ lệ ~ 20% 
    + Phòng ban R&D chiếm tỷ lệ thấp nhất ~14.4%

In [ ]:
fig2 = make_subplots(
    rows=3, cols=3,
    subplot_titles=[f' {cat}' for cat in sorted(df['JobRole'].unique())],  
    vertical_spacing=0.05,
    horizontal_spacing=0.05,
    specs=[[{'type': 'domain'} for _ in range(3)] for _ in range(3)]
)

attrition_values = sorted(df['Attrition'].unique())
display_labels = ['Đang làm việc' if val == 'No' else 'Nghỉ việc' for val in attrition_values] 
colors = ['#66B2FF', '#F30909']  


job_roles = sorted(df['JobRole'].unique())

for idx, role in enumerate(job_roles):
    
    row = (idx // 3) + 1  
    col = (idx % 3) + 1   
    
    field_data = df[df['JobRole'] == role]
    

    attrition_counts = field_data['Attrition'].value_counts()
   
    total = attrition_counts.sum()
    circle = [attrition_counts.get(attr, 0) / total * 100 for attr in attrition_values]
    
    fig2.add_trace(
        go.Pie(
            labels=display_labels,
            values=circle,
            textinfo='percent',
            textposition='inside',
            hole=0,
            marker=dict(colors=colors, line=dict(color='black', width=1)),
            pull=[0.1 if i == 1 else 0 for i in range(len(display_labels))],  
            showlegend=(idx == 0)
        ),
        row=row, col=col
    )


fig2.update_layout(
    width=1800,
    height=600,
    title_text="Tỷ lệ nhân viên nghỉ việc theo Vai trò công việc",
    template='plotly_white'
)

for row in range(1, 4):
    for col in range(1, 4):
        fig2.update_xaxes(showticklabels=False, row=row, col=col)
        fig2.update_yaxes(showticklabels=False, row=row, col=col)

fig2.show()

- Nhận xét:
    - Trong JobRole, có 4 trong tổng số 9 công việc có tỷ lệ nghỉ việc thấp hơn 7% bao gồm: Healthcare, Manager, Research Director và Manufacturing Director, đây là những vị trí có vai trò quan trọng trong công ty và thông thường những nhân viên ở vị trí này có sự ổn định và có thâm niên trong công ty nên xu hướng nghỉ việc của họ thấp.
    - Các vai trò khác có tỷ lệ đều trên 15% trong dó ta có thể thấy ở vai trò Sales Executive và Sales Representative có tỉ lệ nghỉ việc khá cao (17.7% và 41.3%), điều này có thể giải thích vì sao ở Phòng ban này chiếm tỷ lệ nhân viên nghỉ việc cao nhất (đã chỉ ra ở trên)

In [53]:
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=[f'OverTime: {cat}' for cat in sorted(df['OverTime'].unique())],
    horizontal_spacing=0.1,
    specs=[[{'type': 'domain'} for _ in range(2)]]
)


attrition_values = sorted(df['Attrition'].unique()) 
display_labels = ['Đang làm việc' if val == 'No' else 'Nghỉ việc' for val in attrition_values] 
colors = ['#66B2FF', '#F30909'] 


overtime_values = sorted(df['OverTime'].unique())

for idx, overtime in enumerate(overtime_values):

    field_data = df[df['OverTime'] == overtime]
    

    attrition_counts = field_data['Attrition'].value_counts()

    total = attrition_counts.sum()
    circle = [attrition_counts.get(attr, 0) / total * 100 for attr in attrition_values]
    

    fig.add_trace(
        go.Pie(
            labels=display_labels,
            values=circle,
            textinfo='percent',
            textposition='inside',
            hole=0,
            marker=dict(colors=colors, line=dict(color='black', width=1)),
            pull=[0.1 if i == 1 else 0 for i in range(len(display_labels))], 
            showlegend=(idx == 0)
        ),
        row=1, col=idx + 1
    )


fig.update_layout(
    width=800, 
    height=400, 
    title_text="Tỷ lệ nhân viên nghỉ việc theo OverTime",
    template='plotly_white'
)


for col in range(1, 3):
    fig.update_xaxes(showticklabels=False, row=1, col=col)
    fig.update_yaxes(showticklabels=False, row=1, col=col)

fig.show()

- Nhận xét: 
    - Ta nhận thấy đối với những nhân viên không làm thêm giờ tại công ty, xu hướng họ nghỉ việc ít hơn.
    - Đối với những người làm thêm giờ ở công ty, ta nhận thấy xu hướng họ nghỉ việc rất cao, chiếm 1/4 trong tổng số nhân viên có làm thêm giờ. Nguyên nhân có thể đến từ áp lực từ công việc làm thêm, hay giá trị nhận được khi thực hiện làm thêm tại công ty không tương xứng với mức thưởng.